# Question 1

In [1]:
def Q1(x: int, prices: list[int]) -> int:
    n = len(prices)
    buy = [float('-inf')] * n
    sell = [0] * n

    buy[0] = -prices[0] - x
    for i in range(1, n):
        buy[i] = max(buy[i-1], sell[i-1] - prices[i] - x)
        sell[i] = max(sell[i-1], buy[i-1] + prices[i] - x)

    return max(0, sell[-1])
# Sample test 
x = 1
prices = [1, 2, 5, 2, 3, 1]
Q1(x,prices)

2

# Question 2

Question 2

Given a dataframe of stock prices where x[t, i] refers to price of stock i on
the day t. Return the following results:

2a) Dataframe containing the monthly returns of each stock where x[m, i] refers
    to the monthly return of stock i on month m.

2b) Dataframe containing the daily cumulative returns of each stock from the
    past 5 days where x[t, i] refers to the cumulative return of stock i from
    day t-4 to day t.

For each of above parts, please use the same given dataset. You can import any
additional library as you require.


In [2]:
import pandas as pd
import numpy as np

# Please do not modify this dataset
np.random.seed(1)
dataset = pd.DataFrame(
    np.random.randint(200, 300, (260, 5)),
    columns=["A", "B", "C", "D", "E"],
    index=pd.date_range("2023-01-01", "2023-12-31", freq="B"))

# Please begin your answer here
def Q2a(dataset: pd.DataFrame) -> pd.DataFrame:
    # Calculate the daily returns of each stock
    daily_returns = dataset.pct_change()
    
    # Calculate the monthly returns of each stock
    monthly_returns = daily_returns.resample('M').apply(lambda x: (1 + x).prod() - 1)
    
    # Format the index to show only month and year
    monthly_returns.index = monthly_returns.index.strftime('%b-%y')
    
    return monthly_returns

# Calculate the monthly returns and print the result
monthly_returns = Q2a(dataset)
monthly_returns

,A,B,C,D,E
Jan-23,0.139241,-0.047170,0.014706,0.392344,-0.196364
Feb-23,-0.155556,0.212871,-0.032609,-0.085911,0.257919
Mar-23,0.223684,0.106122,-0.198502,0.041353,-0.226619
Apr-23,-0.146953,-0.029520,0.149533,0.043321,-0.060465
May-23,0.163866,-0.114068,-0.170732,-0.173010,0.321782
Jun-23,0.032491,-0.094421,0.299020,0.225941,-0.194757
Jul-23,-0.083916,0.099526,-0.075472,-0.017065,0.125581
Aug-23,-0.160305,0.086207,-0.028571,-0.079861,0.157025
Sep-23,0.154545,-0.150794,0.079832,-0.015094,-0.142857
Oct-23,0.137795,0.186916,-0.206226,-0.068966,-0.120833


In [3]:
def Q2b(dataset: pd.DataFrame) -> pd.DataFrame:
    # Calculate the daily returns of each stock
    daily_returns = dataset.pct_change()
    
    # Calculate the daily cumulative returns of each stock from the past 5 days
    daily_cumulative_returns = daily_returns.rolling(5).apply(lambda x: (1 + x).prod() - 1)
    
    # Remove the first 4 rows as they don't have 5 days of data 
    daily_cumulative_returns = daily_cumulative_returns.iloc[4:]
    
    # Format the index to show date 't'
    daily_cumulative_returns.index = daily_cumulative_returns.index.strftime('%Y-%m-%d')
    
    return daily_cumulative_returns

daily_cumulative_returns = Q2b(dataset)
daily_cumulative_returns[:10]

,A,B,C,D,E
2023-01-06,NaN,NaN,NaN,NaN,NaN
2023-01-09,0.210970,0.386792,0.088235,0.368421,-0.225455
2023-01-10,0.019512,-0.258065,-0.003788,0.208333,0.104478
2023-01-11,-0.068841,-0.258303,-0.029126,0.155556,0.124000
2023-01-12,-0.054545,0.321101,-0.250000,0.170616,0.192982
2023-01-13,0.004367,0.266355,-0.188000,0.007463,-0.229965
2023-01-16,-0.132404,-0.125850,-0.314189,-0.062937,0.051643
2023-01-17,0.162679,0.333333,-0.140684,-0.034483,0.261261
2023-01-18,-0.062257,0.402985,0.075000,0.015385,-0.046263
2023-01-19,0.081731,0.034722,0.347418,-0.161943,-0.169118


# Question 3

In [4]:
import pandas as pd
import numpy as np
model = pd.read_csv('Q3_model_target.csv', index_col=0)
algo1 = pd.read_csv('Q3_algo1.csv')
algo2 = pd.read_csv('Q3_algo2.csv')

## Question 3a

In [5]:
def Q3a(model: pd.DataFrame, algo1: pd.DataFrame, algo2: pd.DataFrame) -> pd.DataFrame:
    # First, calculate the total filled quantity and total filled amount for each trade_id in algo1 and algo2
    algo1['total_filled_amount'] = algo1['filled_qty'] * algo1['filled_price']
    algo2['total_filled_amount'] = algo2['filled_qty'] * algo2['filled_price']

    grouped_algo1 = algo1.groupby('trade_id').agg({'filled_qty': 'sum', 'total_filled_amount': 'sum'})
    grouped_algo2 = algo2.groupby('trade_id').agg({'filled_qty': 'sum', 'total_filled_amount': 'sum'})

    # Then, calculate the average filled price for each trade_id
    grouped_algo1['avg_filled_price'] = grouped_algo1['total_filled_amount'] / grouped_algo1['filled_qty']
    grouped_algo2['avg_filled_price'] = grouped_algo2['total_filled_amount'] / grouped_algo2['filled_qty']

    # Determine the side of the trade for algo1
    grouped_algo1['side'] = np.where(grouped_algo1['avg_filled_price'] > model['target_price'], -1, 1)

    # Now, merge the model dataframe with the grouped_algo1 and grouped_algo2 dataframes
    merged_df = pd.merge(model, grouped_algo1[['avg_filled_price', 'side']], left_index=True, right_index=True, how='left')
    merged_df = pd.merge(merged_df, grouped_algo2['avg_filled_price'], left_index=True, right_index=True, how='left', suffixes=('_algo1', '_algo2'))

    # Finally, rename the columns to match the required output
    merged_df.rename(columns={'avg_filled_price_algo1': 'algo1_price', 'avg_filled_price_algo2': 'algo2_price', 'side': 'side'}, inplace=True)
    
    # The final dataframe merged_df now has the required structure
    return merged_df

df = Q3a(model, algo1, algo2)
df

,order_type,target_price,algo1_price,side,algo2_price
trade_id,,,,,
1,C,292.13,291.855385,1,292.745000
2,B,422.29,424.395200,-1,423.034000
3,C,373.44,372.873571,1,372.692500
4,C,231.93,232.666000,-1,232.992000
5,A,359.25,359.285417,-1,358.773333
...,...,...,...,...,...
996,A,236.48,236.798611,-1,236.561667
997,C,376.71,379.197778,-1,375.904722
998,B,364.30,364.752692,-1,363.397692


In [6]:
df.head()

,order_type,target_price,algo1_price,side,algo2_price
trade_id,,,,,
1,C,292.13,291.855385,1,292.745000
2,B,422.29,424.395200,-1,423.034000
3,C,373.44,372.873571,1,372.692500
4,C,231.93,232.666000,-1,232.992000
5,A,359.25,359.285417,-1,358.773333


## Question 3b

Null Hypothesis (H0): The expected slippages of algo1 and algo2 are the same.
    
Alternative Hypothesis (H1): The expected slippage of algo1 is smaller than algo2

### Check for Normality: 
I use the Shapiro-Wilk test to check if the data is normally distributed. The null hypothesis of this test is that the data is normally distributed. If the p-value is less than 0.05, reject the null hypothesis and conclude that the data is not normally distributed.

In [7]:
from scipy import stats


def checknormality(df: pd.DataFrame):
    # Calculate the slippage for each trade for both algorithms
    df['slippage_algo1'] = df['side'] * (df['target_price'] - df['algo1_price'])
    df['slippage_algo2'] = df['side'] * (df['target_price'] - df['algo2_price'])
    
    # Check if slippage_algo1 is normally distributed
    stat, p_value = stats.shapiro(df['slippage_algo1'])

    if p_value < 0.05:
        print("p_value:",p_value,"so slippage_algo1 is not normally distributed")
    else:
        print("slippage_algo1 is normally distributed")

    # Check if slippage_algo2 is normally distributed
    stat, p_value = stats.shapiro(df['slippage_algo2'])

    if p_value < 0.05:
        print("p_value:",p_value,"so slippage_algo2 is not normally distributed")
    else:
        print("slippage_algo2 is normally distributed")
        
checknormality(df)

p_value: 4.792524698697962e-30 so slippage_algo1 is not normally distributed
p_value: 3.364546271797386e-12 so slippage_algo2 is not normally distributed


Since the slippage data for both algorithms is not normally distributed, the assumptions of the t-test are violated. Therefore, we should use a non-parametric test that does not require the data to be normally distributed.

One such test is the Mann-Whitney U test, also known as the Wilcoxon rank-sum test.

In [8]:
def Q3b(df: pd.DataFrame):
    # Calculate the slippage for each trade for both algorithms
    df['slippage_algo1'] = df['side'] * (df['target_price'] - df['algo1_price'])
    df['slippage_algo2'] = df['side'] * (df['target_price'] - df['algo2_price'])
    # Mann-Whitney U test si
    u_stat, p_value = stats.mannwhitneyu(df['slippage_algo1'], df['slippage_algo2'], alternative='less')
    print(f'U-statistic: {u_stat}')
    print(f'P-value: {p_value}')
    if p_value < 0.05:
        print("We reject the null hypothesis and conclude that the expected slippage of algo1 is smaller than algo2.")
    else:
        print("We fail to reject the null hypothesis and conclude that the expected slippages of algo1 and algo2 are the same.")
Q3b(df)

U-statistic: 757628.0
P-value: 1.0
We fail to reject the null hypothesis and conclude that the expected slippages of algo1 and algo2 are the same.


Based on your Mann-Whitney U test results, the p-value is 1.0, which is greater than the typical significance level of 0.05. This means we fail to reject the null hypothesis.
In other words, there’s not enough evidence to conclude that the expected slippage of algo1 is smaller than algo2. Therefore, we can say that the expected slippages of algo1 and algo2 are the same based on the data you have.

## Question 3c

Based on the results from part 3b, we concluded that there’s not enough evidence to say that one algorithm has a smaller expected slippage than the other. Therefore, you can choose either algorithm for this analysis. Let’s say we choose algo1.

Now, we want to analyze which order_type has the worst expected slippage. We can do this by performing an Analysis of Variance (ANOVA) test. The ANOVA test allows us to compare the means of more than two groups.

Here are our null and alternative hypotheses:

Null Hypothesis (H0): The expected slippages are the same for all order types.

Alternative Hypothesis (H1): At least one order type has a different expected slippage.

In [9]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
def Q3c(df: pd.DataFrame):
    df['slippage_algo1'] = df['side'] * (df['target_price'] - df['algo1_price'])
    df['slippage_algo2'] = df['side'] * (df['target_price'] - df['algo2_price'])
    
    # Perform the ANOVA test
    model = ols('slippage_algo1 ~ C(order_type)', data=df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    print(anova_table)
    
Q3c(df)

                   sum_sq     df          F        PR(>F)
C(order_type)   63.258833    2.0  95.803849  8.766474e-39
Residual       329.157215  997.0        NaN           NaN


In your case, the p-value for C(order_type) is extremely small (8.766474e-39), which is less than 0.05. Therefore, we reject the null hypothesis and conclude that at least one order type has a different expected slippage.

In [10]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Perform Tukey's HSD test
posthoc = pairwise_tukeyhsd(df['slippage_algo1'], df['order_type'], alpha=0.05)

print(posthoc)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj lower  upper  reject
-------------------------------------------------
     A      B   0.3724   0.0 0.2617 0.4832   True
     A      C   0.6109   0.0 0.5073 0.7146   True
     B      C   0.2385   0.0 0.1362 0.3408   True
-------------------------------------------------


The results from Tukey’s HSD (Honestly Significant Difference) test show that the mean slippage is significantly different between all pairs of order types (A, B, and C).

Here’s a summary of the results:

Order type A vs B: The mean difference in slippage is 0.3724, and this difference is statistically significant (p < 0.05). This means that the expected slippage for order type B is higher than for order type A.

Order type A vs C: The mean difference in slippage is 0.6109, and this difference is statistically significant (p < 0.05). This means that the expected slippage for order type C is higher than for order type A.

Order type B vs C: The mean difference in slippage is 0.2385, and this difference is statistically significant (p < 0.05). This means that the expected slippage for order type C is higher than for order type B.

Based on these results, we can conclude that order type C has the worst expected slippage, followed by order type B, and then order type A.

## Question 3d